In [1]:
import sklearn
import sklearn.linear_model
import sklearn.neural_network
import sklearn.metrics
import chess
import tqdm
import pandas as pd
import numpy as np
from copy import deepcopy
import time

In [2]:
base = np.zeros(64)
inf = 10000

In [3]:
def sq(ind):
    return 7 - ind // 8, ind % 8

In [4]:
df = pd.read_csv("./data/random_evals.csv")

In [5]:
def get_data(fen):
    
    board = chess.Board(fen)
    
    w_pawns = deepcopy(base)
    
    for i in board.pieces(chess.PAWN, chess.WHITE):
        w_pawns[i] = 1
    
    b_pawns = deepcopy(base)
    
    for i in board.pieces(chess.PAWN, chess.BLACK):
        b_pawns[i] = 1
    
    w_pieces = deepcopy(base)
    
    for i in range(64):
        if board.piece_at(i) and board.piece_at(i).color is chess.WHITE:
            w_pieces[i] = 1
    
    b_pieces = deepcopy(base)
    
    for i in range(64):
        if board.piece_at(i) and board.piece_at(i).color is chess.BLACK:
            b_pieces[i] = 1
    
    w_king = deepcopy(base)
    
    for i in board.pieces(chess.KING, chess.WHITE):
        w_king[i] = 1
    
    b_king = deepcopy(base)
    
    for i in board.pieces(chess.KING, chess.BLACK):
        b_king[i] = 1
    
    w_attacks = deepcopy(base)
    
    for i in range(64):
        w_attacks[i] = len(board.attackers(chess.WHITE, i)) / 16
    
    b_attacks = deepcopy(base)
    
    for i in range(64):
        b_attacks[i] = len(board.attackers(chess.BLACK, i)) / 16
    
    return np.concatenate((w_pawns, b_pawns, w_pieces, b_pieces, w_king, b_king, w_attacks, b_attacks))
    

In [6]:
df = df[:100000]
n = len(df)
x, y = [], []
mx = 0

In [7]:
for it in tqdm.tqdm(range(n)):
    x.append(get_data(df['FEN'][it]))
    # df['FEN'][it] = get_data(df['FEN'][it])
    if "#-" in df['Evaluation'][it]:
        # df['Evaluation'][it] = -inf
        y.append(-inf / 100)
    elif "#" in df['Evaluation'][it]:
        # df['Evaluation'][it] = inf
        y.append(inf / 100)
    else:
        mx = max(mx, abs(int(df['Evaluation'][it])))
        y.append(int(df['Evaluation'][it]) / 100)
        # df['Evaluation'][it] = int(df['Evaluation'][it])

100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [02:30<00:00, 663.52it/s]


In [8]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size = 0.25, random_state = 42)

In [11]:
model = sklearn.neural_network.MLPRegressor(max_iter = 1000, random_state = 42, hidden_layer_sizes = (512, 512, 512, 512, 512), verbose = True, tol = 0.000001, n_iter_no_change = 100)
model.fit(x_train, y_train)

Iteration 1, loss = 173.95932286
Iteration 2, loss = 126.74108547
Iteration 3, loss = 105.08525327
Iteration 4, loss = 90.19427478
Iteration 5, loss = 79.50289328
Iteration 6, loss = 72.92797421
Iteration 7, loss = 65.87317065
Iteration 8, loss = 59.99010822
Iteration 9, loss = 55.28502005
Iteration 10, loss = 50.87363234
Iteration 11, loss = 48.16744586
Iteration 12, loss = 46.02681456
Iteration 13, loss = 42.32746533
Iteration 14, loss = 38.81196334
Iteration 15, loss = 36.93263508
Iteration 16, loss = 34.67241129
Iteration 17, loss = 34.63448295
Iteration 18, loss = 31.64833123
Iteration 19, loss = 29.30346962
Iteration 20, loss = 28.65426486
Iteration 21, loss = 28.70541724
Iteration 22, loss = 28.75260514
Iteration 23, loss = 26.79788110
Iteration 24, loss = 24.97030199
Iteration 25, loss = 24.32536757
Iteration 26, loss = 22.27243551
Iteration 27, loss = 21.92625047
Iteration 28, loss = 23.06841222
Iteration 29, loss = 20.38609210
Iteration 30, loss = 19.38565491
Iteration 31, lo

MLPRegressor(hidden_layer_sizes=(512, 512, 512, 512, 512), max_iter=1000,
             n_iter_no_change=100, random_state=42, tol=1e-06, verbose=True)

In [12]:
print(model.score(x_test, y_test))
print(sklearn.metrics.mean_squared_error(y_test, model.predict(x_test)))
print(sklearn.metrics.mean_absolute_error(y_test, model.predict(x_test)))

0.7274602526563709
156.0329133269711
4.025079360897066


In [42]:
def moves(fen):
    board = chess.Board(fen)
    ar = []
    z = board.turn * 2 - 1
    for move in board.legal_moves:
        board.push(move)
        ar.append([z * model.predict([get_data(board.fen())]), str(move)])
        board.pop()
    ar = sorted(ar)
    for i in ar[::-1]:
        print(i[0], i[1])

In [43]:
moves('rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R b KQkq - 1 1')

[0.49337248] e7e5
[-0.06584847] d7d5
[-0.53083753] g8f6
[-0.53083753] c7c6
[-0.53083753] c7c5
[-0.53083753] b8c6
[-0.76779175] e7e6
[-0.79070188] d7d6
[-1.16970656] f7f5
[-1.21895799] h7h6
[-1.22243606] a7a6
[-1.2677208] g7g6
[-1.41020857] g8h6
[-1.55666756] f7f6
[-1.58321759] b7b6
[-1.61090128] b8a6
[-1.66448677] h7h5
[-1.72145884] a7a5
[-2.14923954] b7b5
[-2.58264387] g7g5


In [48]:
tt = time.time()
for i in range(1000):
    model.predict([get_data('rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R b KQkq - 1 1')])
print(time.time() - tt)

3.434262752532959


In [14]:
from joblib import dump, load
dump(model, 'model.joblib')

['model.joblib']